In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('../../data/df_sen_sub/train.csv')
data = data.fillna('')
senti = list(data['sentiment_value'])
subs = list(data['subject'])
content = list(data['content'])
test_data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])

In [5]:
len(data[data['sentiment_value'] == -1]), len(data[data['sentiment_value'] == 1]), len(data[data['sentiment_value'] == 0])

(1616, 1670, 6661)

In [4]:
with open('../../data/df_sen_sub/add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs)))
# 迭代器，使用jieba将句子进行分词
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                yield list(jieba.cut(line))

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    #model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    #model.save("../../data/df_sen_sub/add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../data/df_sen_sub/add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../data/df_sen_sub/add_word2vec_model")
sentences = list(sentences)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.961 seconds.
Prefix dict has been built succesfully.


In [5]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(sentences) - len(content)):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    else:
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])     
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

y_map = []
map_ = [0, -1, 1]
y = list(data['sentiment_value'])
for i in range(len(y)):
    y_map.append(map_.index(y[i]))
y_map = np.array(y_map)
y_map.shape


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(9947, 50, 100) (2364, 50, 100)


(9947,)

In [6]:
X = x_vecs
X_test = x_test_vecs
y = y_map
pretrained = pretrained

In [7]:
import pickle
pickle.dump(pretrained, open('../../data/pretrained.txt', 'wb'))
pickle.dump(word2ind, open('../../data/word2ind.txt', 'wb'))
pickle.dump(senti_test_sen, open('../../data/senti_test_sen.txt', 'wb'))
pickle.dump(senti_sen, open('../../data/senti_sen.txt', 'wb'))
pickle.dump(X, open('../../data/sentiment_x.txt', 'wb'))
pickle.dump(X_test, open('../../data/sentiment_x_test.txt', 'wb'))
pickle.dump(y, open('../../data/sentiment_y.txt', 'wb'))

In [3]:
import pickle
word2ind = pickle.load(open('../../data/word2ind.txt', 'rb'))
senti_test_sen = pickle.load(open('../../data/senti_test_sen.txt', 'rb'))
senti_sen = pickle.load(open('../../data/senti_sen.txt', 'rb'))
X = pickle.load(open('../../data/sentiment_x.txt', 'rb'))
X_test = pickle.load(open('../../data/sentiment_x_test.txt', 'rb'))
y = pickle.load(open('../../data/sentiment_y.txt', 'rb'))
pretrained = pickle.load(open('../../data/pretrained.txt', 'rb'))

In [4]:
set_subs = list(set(subs))
va = []
for i in range(len(set_subs)):
    print(set_subs[i], word2ind[set_subs[i]])
    va.append([word2ind[set_subs[i]]])

安全性 1333
空间 2102
油耗 731
舒适性 1156
内饰 1220
外观 2203
动力 805
操控 1427
配置 294
价格 18


In [5]:
pretrained.append([0.0] * 100)
word2ind['null'] = 17387

In [6]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [7]:
train_x, test_x, train_y, test_y = train_test_split(han_x, y, test_size=0.2, random_state=42)
#train_x = han_x
#train_y = y
#test_x = han_x_test

In [8]:
len(train_x), len(train_y), len(test_x)

(7957, 7957, 1990)

In [9]:
trainset = [(train_x[i], train_y[i]) for i in range(len(train_x))]
testset = [(test_x[i], test_y[i]) for i in range(len(test_x))]

In [10]:
trainloader = torch.utils.data.DataLoader(
                    trainset, 
                    batch_size=4,
                    shuffle=True, 
                    num_workers=2)
testloader = torch.utils.data.DataLoader(
                    testset, 
                    batch_size=4,
                    shuffle=False, 
                    num_workers=2)

In [11]:
torch.ones((50, 1)).reshape((1,2,25))

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1.]]])

In [47]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(100, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x) #[4,50,100]
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        t_va = t_va.reshape((-1, 100)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr))
        x = self.fc3(final)         
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)

Net(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=100, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (embeddings): Embedding(19791, 100)
)


In [ ]:
torch.set_num_threads(8)
for epoch in range(30):  
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # 输入数据
        inputs, labels = data
        # 梯度清零
        optimizer.zero_grad()
        # forward + backward 
        va = torch.from_numpy(np.array(va))
        outputs = net(inputs, va)
        loss = criterion(outputs, labels)
        loss.backward()   
        
        # 更新参数 
        optimizer.step()
        
        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 200 == 199: # 每200个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 200))
            running_loss = 0.0
print('Finished Training')


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 0.918
[1,   400] loss: 0.865
[1,   600] loss: 0.870
[1,   800] loss: 0.882
[1,  1000] loss: 0.773
[1,  1200] loss: 0.786
[1,  1400] loss: 0.792
[1,  1600] loss: 0.795
[1,  1800] loss: 0.792
[2,   200] loss: 0.735
[2,   400] loss: 0.797
[2,   600] loss: 0.792
[2,   800] loss: 0.779
[2,  1000] loss: 0.766
[2,  1200] loss: 0.780
[2,  1400] loss: 0.830
[2,  1600] loss: 0.775
[2,  1800] loss: 0.769
[3,   200] loss: 0.795
[3,   400] loss: 0.717
[3,   600] loss: 0.751
[3,   800] loss: 0.767
[3,  1000] loss: 0.724
[3,  1200] loss: 0.762
[3,  1400] loss: 0.745
[3,  1600] loss: 0.760
[3,  1800] loss: 0.704
[4,   200] loss: 0.691
[4,   400] loss: 0.717
[4,   600] loss: 0.657
[4,   800] loss: 0.644
[4,  1000] loss: 0.651
[4,  1200] loss: 0.568
[4,  1400] loss: 0.637
[4,  1600] loss: 0.597
[4,  1800] loss: 0.585
[5,   200] loss: 0.502
[5,   400] loss: 0.445
[5,   600] loss: 0.503
[5,   800] loss: 0.463
[5,  1000] loss: 0.436
[5,  1200] loss: 0.462
[5,  1400] loss: 0.457
[5,  1600] 

In [56]:
map_ = [0, -1, 1]
dataiter = iter(testloader)
images, labels = dataiter.next() # 一个batch返回4张图片q
images = dataiter.next() # 一个batch返回4张图片
print('实际的label: ', ' '.join(\
            '%08s'%map_[labels[j]] for j in range(4)))
# 计算图片在每个类别上的分数
outputs = net(images,va)
# 得分最高的那个类
_, predicted = torch.max(outputs.data, 1)

print('预测结果: ', ' '.join('%5s'\
            % map_[predicted[j]] for j in range(4)))

实际的label:         1        0        0        0


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [55]:
correct = 0 # 预测正确的图片数
total = 0 # 总共的图片数


# 由于测试的时候不需要求导，可以暂时关闭autograd，提高速度，节约内存
fake = {}
text = []
vec = []
l = []
p = []
res = []
map_ = [0, -1, 1]
with torch.no_grad():
    for data in testloader:
        images = data#images, labels = data
        outputs = net(images,va)
        _, predicted = torch.max(outputs, 1)
        res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
        #for j in range(len(predicted)):
            #if predicted[j] != labels[j]:
                #vec.append(images[j])
                #l.append(map_[int(labels[j])])
                #p.append(map_[int(predicted[j])])
        #total += labels.size(0)
        #correct += (predicted == labels).sum()

#print('3283张测试集中的准确率为: %d %%' % (100 * correct / total))

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f25a9108ef0>>
Traceback (most recent call last):
  File "/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/multiprocessing/queues.py", line 345, in get
    return _ForkingPickler.loads(res)
  File "/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/hom

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [ ]:
ind2word = {}
for key in word2ind:
    ind2word[word2ind[key]] = key

In [ ]:
ind2word[0]

In [ ]:
for i in range(len(vec)):
    temp = vec[i]
    tt = []
    for j in range(len(temp)):
        tt.append(ind2word[int(temp[j])])
    text.append(''.join(tt))

In [ ]:
len(res)

In [ ]:
fake = {}
content_id,subject,sentiment_value,sentiment_word
fake['text'] = test_content
fake['sentiment_value'] = res
df = pd.DataFrame(fake)
df.to_csv('../../data/df_sen_sub/senti_result.csv')

In [ ]:
df[df['predict'] == 0]